In [8]:
#_*_coding: utf-8_*_
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import DOUBLE_PRECISION
import sqlalchemy
import psycopg2
import time
import os

In [9]:
conn_string = os.environ['PG_DB']
cur_path = os.getcwd()
ser = Service(cur_path+'/chromedriver')

In [10]:
df_k = pd.read_excel("keywords.xlsx", sheet_name=None)
list_word=[]
for project in df_k.keys():
    #display(df_k[project]['Статистика по словам'][0])
    list_word.append(df_k[project]['Статистика по словам'][0])
#display(list_word)
print(len(list_word))

1


In [11]:
options = webdriver.ChromeOptions()
# get source code
df_list=[]
for keyword in list_word:
    options.add_argument("--user-data-dir="+cur_path+"/profile")
    browser=webdriver.Chrome(service=ser, options=options)
    #https://wordstat.yandex.ru/#!/history?words=простоквашино
    browser.get(r'https://wordstat.yandex.ru/#!/history?words='+keyword)
    time.sleep(15)
    html = browser.page_source
    time.sleep(1)
    browser.close()
    dfs = pd.read_html(html)
    df_1 = dfs[8]
    df_1 = df_1.drop(df_1.columns[[1,3]], axis = 1)
    df_1 = df_1.drop(0, axis = 0)
    df_2 = dfs[10]
    df_2 = df_2.drop(df_2.columns[[1,3]], axis = 1)
    df_2 = df_2.drop(0, axis = 0)
    df = pd.concat([df_1, df_2])
    df = df.reset_index()
    df["request"] = keyword
    df = df.drop(df.columns[[0]], axis = 1)
    df_list.append(df)
df_all=pd.concat(df_list)

In [12]:
df_all['date_from']= df_all['Период'].str.split('-', n=1, expand=True)[0].str.strip()
df_all['date_to']= df_all['Период'].str.split('-', n=1, expand=True)[1].str.strip()
df_all.date_from = pd.to_datetime(df_all.date_from,  format = '%d.%m.%Y')
df_all.date_to = pd.to_datetime(df_all.date_to, format = '%d.%m.%Y')

In [13]:
df_all = df_all.reset_index()

In [14]:
display(df_all)

,index,Период,Абсолютное,request,date_from,date_to
0,0,01.02.2021 - 28.02.2021,150286.0,оранжевая корова,2021-02-01,2021-02-28
1,1,01.03.2021 - 31.03.2021,158857.0,оранжевая корова,2021-03-01,2021-03-31
2,2,01.04.2021 - 30.04.2021,161684.0,оранжевая корова,2021-04-01,2021-04-30
3,3,01.05.2021 - 31.05.2021,155542.0,оранжевая корова,2021-05-01,2021-05-31
4,4,01.06.2021 - 30.06.2021,150960.0,оранжевая корова,2021-06-01,2021-06-30
5,5,01.07.2021 - 31.07.2021,149574.0,оранжевая корова,2021-07-01,2021-07-31
6,6,01.08.2021 - 31.08.2021,139875.0,оранжевая корова,2021-08-01,2021-08-31
7,7,01.09.2021 - 30.09.2021,112064.0,оранжевая корова,2021-09-01,2021-09-30
8,8,01.10.2021 - 31.10.2021,112815.0,оранжевая корова,2021-10-01,2021-10-31
9,9,01.11.2021 - 30.11.2021,112587.0,оранжевая корова,2021-11-01,2021-11-30


In [13]:
db = create_engine(conn_string)
conn = db.connect()

In [14]:
print("Reading yt_reels_stat table with latest date...")
df_old =  pd.read_sql("SELECT * FROM wordstat_monthly", conn, parse_dates=['date_from', 'date_to'])
print("Done.")

Reading yt_reels_stat table with latest date...
Done.


In [58]:
df_res = pd.concat([df_old, df_all])

In [65]:
df_res = df_res.drop_duplicates(subset = ['Период', 'request'], keep = 'last') 

In [68]:
#display(df_res)

In [11]:
inject_dict = { 'Период': sqlalchemy.types.TEXT(),  
                'Абсолютное': sqlalchemy.types.BIGINT(), 
                'request': sqlalchemy.types.TEXT(),  
                'date_from': sqlalchemy.types.Date(),                   
                'date_to': sqlalchemy.types.Date(), 
                 }
start_time = time.time()
res = df_all.to_sql('wordstat_monthly', con=conn, if_exists='replace', index=False, dtype = inject_dict)
print("to_sql duration: {} seconds".format(time.time() - start_time))

to_sql duration: 1.855560064315796 seconds
